# 08. Синтетична валідація — перевірка всього pipeline

До 15% вибірки 2023 року (~524K тендерів) додаються ~480 штучних тендерів та ~1050 ставок (0.09% від загального обсягу). Враховуються лише ті правила та статистичні тести, що є релевантними до сконструйованого патерну (побічні спрацювання не зараховуються).

## 7 сценаріїв аномалій

| # | Сценарій | Що генеруємо | Що повинно ловити |
|---|----------|-------------|-------------------|
| 1a | **Простий картель** | 5 постачальників по черзі перемагають у 50 тендерах, ставки підозріло близькі (CV < 3%) | Статистика (cv_anomaly, bid_clustering), мережа (ротація), правила (close_to_winner — часткове) |
| 1b | **Складний картель** | Ті ж 5 постачальників, ще 50 тендерів, але участь ~70% ймовірністю (2-5 учасників), розкид ставок нормальний (2-35%) | Тільки мережа (ротація перемог) — статистика і правила не бачать |
| 2 | **Монополізація** | Замовник спрямовує 70% контрактів одному постачальнику, 50% без конкуренції (80 тендерів) | Правила (buyer_supplier_dominance — часткове), ML (аномальний портрет) |
| 3 | **Маніпуляція цінами** | Публікація у вихідні, круглі ціни, знижки 50–80% (60 тендерів) | Правила (weekend_publication, extreme_discount), статистика (zscore_discount, round_price) |
| 4 | **Дроблення контрактів** | 150 тендерів на суми 190–199K (нижче порогу 200K), один постачальник, одна CPV | Правила (threshold_manipulation, same_day_same_supplier), статистика (high_concentration, monopoly), ML (портрет), мережа (монополістична пара) |
| 5 | **Ексклюзивна пара** | 30 тендерів на 10–50M UAH, єдиний учасник, 100% ексклюзивність | Усі 5 методів: правила, статистика, IForest, LOF, мережа |
| 6 | **Замаскована монополізація** | H1 (30 тендерів): різні постачальники, конкурентні. H2 (30 тендерів): 80% одному постачальнику, 60% без конкуренції. Річний портрет маскує концентрацію H2 | Правила (часткове — лише тендери H2), ML (залежить від агрегації) |

In [1]:
# Cell 1: Imports & Load Real Data
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from src.data_loader import load_tenders, load_bids, load_buyers
from src.detectors import (
    RuleBasedDetector, StatisticalDetector,
    AggregatedPyOD, NetworkAnalysisDetector
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Load real data (2023 sample for manageable size)
print("Loading real tenders...")
real_tenders = load_tenders(years=[2023], sample_frac=0.15, random_state=RANDOM_STATE)
print(f"Real tenders: {len(real_tenders):,}")

print("\nLoading real bids...")
real_bids = load_bids(years=[2023], sample_frac=0.15, random_state=RANDOM_STATE)
print(f"Real bids: {len(real_bids):,}")

print("\nLoading buyers...")
buyers = load_buyers()
print(f"Buyers: {len(buyers):,}")

# Get percentiles from real data for realistic synthetic generation
value_p50 = real_tenders['tender_value'].median()
value_p95 = real_tenders['tender_value'].quantile(0.95)
value_p99 = real_tenders['tender_value'].quantile(0.99)
print(f"\nReal data value percentiles: P50={value_p50:,.0f}, P95={value_p95:,.0f}, P99={value_p99:,.0f}")

Loading real tenders...
Scanning 2023...
Sampled to 523,762 records (15%)
Real tenders: 523,762

Loading real bids...
Scanning bids 2023...
Loaded 73,306 bids
Real bids: 73,306

Loading buyers...
Loaded buyers: 35,995
Buyers: 35,995

Real data value percentiles: P50=4,865, P95=323,419, P99=2,600,000


In [2]:
# Cell 2: Generate Synthetic Tenders & Bids

def generate_synthetic_tenders():
    """Generate ~480 synthetic tenders + bids across 7 anomaly scenarios."""
    all_tenders = []
    all_bids = []
    tender_counter = 0
    bid_counter = 0
    
    # Common CPV codes from real data
    common_cpv = 33.0  # Medical equipment
    
    def make_tender(buyer_id, supplier_id, tender_value, award_value,
                    n_tenderers, is_single, is_competitive, method,
                    is_weekend=0, is_q4=0, is_december=0, cpv=None,
                    has_enquiries=1, award_criteria='lowestCost',
                    published_date=None, scenario=''):
        nonlocal tender_counter
        tender_counter += 1
        tid = f'SYNTH_T_{tender_counter:05d}'
        
        price_change_pct = ((tender_value - award_value) / tender_value * 100) if tender_value > 0 else 0
        
        if published_date is None:
            # Random date in 2023
            day_offset = np.random.randint(0, 365)
            published_date = pd.Timestamp('2023-01-01') + pd.Timedelta(days=int(day_offset))
        
        # Add small noise to values (±5%) to avoid identical records
        noise = np.random.uniform(0.95, 1.05)
        tender_value_noisy = tender_value * noise
        award_value_noisy = award_value * noise
        
        return {
            'tender_id': tid,
            'ocid': f'ocds-synth-{tid}',
            'buyer_id': buyer_id,
            'procuring_entity_id': buyer_id,
            'supplier_id': supplier_id,
            'locality': 'SYNTH_CITY',
            'postal_code': '00000',
            'procurement_method': method,
            'main_procurement_category': 'goods',
            'award_criteria': award_criteria,
            'main_cpv_code': f'{int(cpv or common_cpv)}000000',
            'currency': 'UAH',
            'year': 2023,
            'month': published_date.month,
            'quarter': (published_date.month - 1) // 3 + 1,
            'day_of_week': published_date.dayofweek,
            'is_q4': is_q4,
            'is_december': is_december,
            'is_weekend': is_weekend,
            'is_single_bidder': is_single,
            'is_competitive': is_competitive,
            'is_cross_region': 0,
            'has_enquiries': has_enquiries,
            'is_buyer_masked': 0,
            'is_supplier_masked': 0,
            'has_multiple_awards': 0,
            'has_unsuccessful_awards': 0,
            'has_cancelled_awards': 0,
            'number_of_items': np.random.randint(1, 5),
            'number_of_tenderers': n_tenderers,
            'number_of_bids': n_tenderers,
            'number_of_awards': 1,
            'number_of_contracts': 1,
            'number_of_documents': np.random.randint(1, 10),
            'active_awards_count': 1,
            'main_cpv_2_digit': cpv or common_cpv,
            'main_cpv_4_digit': (cpv or common_cpv) * 100 + 10,
            'tender_value': tender_value_noisy,
            'award_value': award_value_noisy,
            'price_change_amount': tender_value_noisy - award_value_noisy,
            'price_change_pct': price_change_pct,
            'award_value_total': award_value_noisy,
            'award_value_max': award_value_noisy,
            'award_value_min': award_value_noisy,
            'award_value_mean': award_value_noisy,
            'award_value_std': 0.0,
            'award_concentration': 1.0,
            'discount_percentage_avg': price_change_pct,
            'discount_percentage_max': price_change_pct,
            'published_date': published_date,
            'award_date': published_date + pd.Timedelta(days=np.random.randint(7, 30)),
            # Synthetic tracking
            'is_synthetic': True,
            'scenario': scenario,
        }
    
    def make_bid(tender_id, bidder_id, bid_amount, is_winner, bid_status='active'):
        nonlocal bid_counter
        bid_counter += 1
        return {
            'tender_id': tender_id,
            'bid_id': f'SYNTH_B_{bid_counter:05d}',
            'bidder_id': bidder_id,
            'bid_status': bid_status,
            'bid_amount': bid_amount,
            'is_winner': int(is_winner),
            'is_bidder_masked': 0,
            'bid_date': pd.Timestamp('2023-06-15'),
        }
    
    # =========================================================================
    # Scenario 1a: Naive Cartel (simple bid rigging)
    # 5 suppliers rotate winning 50 tenders, ALL 5 bid every time
    # Bids suspiciously close (CV < 3%) — detectable by statistics
    # =========================================================================
    print("Generating Scenario 1a: Naive Cartel...")
    cartel_buyer = 'buyer_SYNTH_001'
    cartel_suppliers = [f'supplier_SYNTH_{i:03d}' for i in range(1, 6)]
    
    for i in range(50):
        winner_idx = i % 5  # Rotate winner
        winner = cartel_suppliers[winner_idx]
        base_value = np.random.uniform(500_000, 2_000_000)
        
        t = make_tender(
            buyer_id=cartel_buyer, supplier_id=winner,
            tender_value=base_value, award_value=base_value * 0.97,
            n_tenderers=5, is_single=0, is_competitive=1,
            method='open', cpv=45.0,  # Construction
            scenario='1a_cartel_naive'
        )
        all_tenders.append(t)
        
        # Create 5 bids with CV < 3% (suspiciously close)
        winning_bid = base_value * 0.97
        for j, supplier in enumerate(cartel_suppliers):
            if supplier == winner:
                bid_amt = winning_bid
                is_win = True
            else:
                # Bids within 0.5-2.5% of winner (suspiciously close)
                bid_amt = winning_bid * np.random.uniform(1.005, 1.025)
                is_win = False
            
            all_bids.append(make_bid(t['tender_id'], supplier, bid_amt, is_win))
    
    # =========================================================================
    # Scenario 1b: Sophisticated Cartel
    # Same 5 suppliers rotate winning 50 more tenders, BUT:
    # - Each non-winner participates with ~70% probability (2-5 bidders)
    # - Bid spread is normal competitive (2-35% above winner)
    # Key signal: rotation in network, NOT bid clustering
    # =========================================================================
    print("Generating Scenario 1b: Sophisticated Cartel...")
    
    for i in range(50):
        winner_idx = i % 5  # Still rotate winner
        winner = cartel_suppliers[winner_idx]
        base_value = np.random.uniform(500_000, 2_000_000)
        
        # Determine which non-winners participate (each with 70% prob)
        participants = [winner]
        for supplier in cartel_suppliers:
            if supplier != winner and np.random.random() < 0.7:
                participants.append(supplier)
        
        # Ensure at least 2 participants (cartel needs a cover bidder)
        if len(participants) < 2:
            remaining = [s for s in cartel_suppliers if s not in participants]
            participants.append(np.random.choice(remaining))
        
        n_tend = len(participants)
        
        t = make_tender(
            buyer_id=cartel_buyer, supplier_id=winner,
            tender_value=base_value, award_value=base_value * 0.97,
            n_tenderers=n_tend, is_single=0, is_competitive=1,
            method='open', cpv=45.0,  # Construction
            scenario='1b_cartel_sophisticated'
        )
        all_tenders.append(t)
        
        # Create bids with normal competitive spread
        winning_bid = base_value * 0.97
        for supplier in participants:
            if supplier == winner:
                bid_amt = winning_bid
                is_win = True
            else:
                # Wide spread: 2-35% above winner (looks like normal competition)
                bid_amt = winning_bid * np.random.uniform(1.02, 1.35)
                is_win = False
            
            all_bids.append(make_bid(t['tender_id'], supplier, bid_amt, is_win))
    
    # =========================================================================
    # Scenario 2: Monopolist Buyer
    # buyer_SYNTH_002 gives 70% to supplier_SYNTH_006, 50% single bidder
    # Moderate dominance — harder to detect than extreme 90%/85%
    # =========================================================================
    print("Generating Scenario 2: Monopolist Buyer...")
    mono_buyer = 'buyer_SYNTH_002'
    mono_supplier = 'supplier_SYNTH_006'
    
    for i in range(80):
        if i < 56:  # 70% to main supplier
            supplier = mono_supplier
        else:
            supplier = f'supplier_SYNTH_OTHER_{i}'
        
        if i < 40:  # 50% single bidder
            n_tend = 1
            is_single = 1
            is_comp = 0
            method = 'limited'
        else:
            n_tend = np.random.randint(2, 4)
            is_single = 0
            is_comp = 1
            method = 'open'
        
        base_val = np.random.uniform(100_000, 500_000)
        discount = np.random.uniform(0.01, 0.05)  # Low discount
        
        t = make_tender(
            buyer_id=mono_buyer, supplier_id=supplier,
            tender_value=base_val, award_value=base_val * (1 - discount),
            n_tenderers=n_tend, is_single=is_single, is_competitive=is_comp,
            method=method, cpv=33.0,  # Medical
            has_enquiries=0,
            scenario='2_monopolist_buyer'
        )
        all_tenders.append(t)
        
        # Create bids
        all_bids.append(make_bid(t['tender_id'], supplier, base_val * (1 - discount), True))
        if n_tend > 1:
            for k in range(n_tend - 1):
                all_bids.append(make_bid(
                    t['tender_id'], f'supplier_random_{i}_{k}',
                    base_val * np.random.uniform(1.0, 1.2), False
                ))
    
    # =========================================================================
    # Scenario 3: Weekend/Night Submitter + Price Manipulator
    # buyer_SYNTH_003: weekend publications, round prices, extreme discounts
    # =========================================================================
    print("Generating Scenario 3: Weekend/Price Manipulator...")
    weekend_buyer = 'buyer_SYNTH_003'
    weekend_suppliers = ['supplier_SYNTH_007', 'supplier_SYNTH_008']
    
    # Find weekend dates in 2023
    all_dates_2023 = pd.date_range('2023-01-01', '2023-12-31')
    weekend_dates = all_dates_2023[all_dates_2023.dayofweek >= 5]  # Saturday=5, Sunday=6
    
    for i in range(60):
        supplier = weekend_suppliers[i % 2]
        pub_date = weekend_dates[i % len(weekend_dates)]
        
        # Round prices (100K, 200K, 500K, etc.)
        round_values = [100_000, 200_000, 300_000, 500_000, 1_000_000]
        base_val = round_values[i % len(round_values)]
        
        # Extreme discount: 50-80%
        discount_pct = np.random.uniform(50, 80)
        award_val = base_val * (1 - discount_pct / 100)
        
        t = make_tender(
            buyer_id=weekend_buyer, supplier_id=supplier,
            tender_value=base_val, award_value=award_val,
            n_tenderers=2, is_single=0, is_competitive=1,
            method='open', is_weekend=1,
            cpv=9.0,  # Fuel
            published_date=pub_date,
            scenario='3_weekend_price_manipulator'
        )
        # Override noise for round values — keep them perfectly round for R023
        t['tender_value'] = float(base_val)
        t['award_value'] = float(award_val)
        t['price_change_pct'] = discount_pct
        t['discount_percentage_avg'] = discount_pct
        t['discount_percentage_max'] = discount_pct
        all_tenders.append(t)
        
        # 2 bids: winner + loser
        all_bids.append(make_bid(t['tender_id'], supplier, award_val, True))
        all_bids.append(make_bid(
            t['tender_id'], weekend_suppliers[(i + 1) % 2],
            base_val * np.random.uniform(0.6, 0.9), False
        ))
    
    # =========================================================================
    # Scenario 4: Contract Splitter
    # buyer_SYNTH_004: 150 tenders just below 200K threshold, same supplier/CPV
    # =========================================================================
    print("Generating Scenario 4: Contract Splitter...")
    splitter_buyer = 'buyer_SYNTH_004'
    splitter_supplier = 'supplier_SYNTH_009'
    
    # Concentrate tenders on same days to trigger R011 contract_splitting
    split_dates = pd.date_range('2023-03-01', periods=30, freq='7D')  # 30 weeks
    
    for i in range(150):
        # Values between 190K-199K (just below 200K threshold)
        base_val = np.random.uniform(190_000, 199_000)
        discount = np.random.uniform(0.005, 0.02)  # Very low discount
        
        # Publish in batches of 5 on same day
        pub_date = split_dates[i % len(split_dates)]
        
        t = make_tender(
            buyer_id=splitter_buyer, supplier_id=splitter_supplier,
            tender_value=base_val, award_value=base_val * (1 - discount),
            n_tenderers=1, is_single=1, is_competitive=0,
            method='limited', cpv=45.0,  # Construction
            has_enquiries=0,
            published_date=pub_date,
            scenario='4_contract_splitter'
        )
        # Keep values near threshold precisely (override noise)
        t['tender_value'] = base_val
        t['award_value'] = base_val * (1 - discount)
        all_tenders.append(t)
        
        # Single bid
        all_bids.append(make_bid(t['tender_id'], splitter_supplier, t['award_value'], True))
    
    # =========================================================================
    # Scenario 5: Exclusive High-Value Pair
    # buyer_SYNTH_005 + supplier_SYNTH_010: 30 very high-value, single bidder
    # =========================================================================
    print("Generating Scenario 5: Exclusive High-Value Pair...")
    exclusive_buyer = 'buyer_SYNTH_005'
    exclusive_supplier = 'supplier_SYNTH_010'
    
    for i in range(30):
        base_val = np.random.uniform(10_000_000, 50_000_000)  # 10M-50M UAH
        discount = np.random.uniform(0.001, 0.01)  # Tiny discount (<1%)
        
        t = make_tender(
            buyer_id=exclusive_buyer, supplier_id=exclusive_supplier,
            tender_value=base_val, award_value=base_val * (1 - discount),
            n_tenderers=1, is_single=1, is_competitive=0,
            method='limited', cpv=72.0,  # IT services
            has_enquiries=0,
            scenario='5_exclusive_high_value'
        )
        all_tenders.append(t)
        
        # Single bid
        all_bids.append(make_bid(t['tender_id'], exclusive_supplier, t['award_value'], True))
    
    # =========================================================================
    # Scenario 6: Gradual Capture (Masked Monopolization)
    # buyer_SYNTH_006: normal in H1 (diverse suppliers, competitive),
    # concentrated in H2 (80% to one supplier, 60% single bidder)
    # Annual portrait masks the H2 concentration
    # =========================================================================
    print("Generating Scenario 6: Gradual Capture...")
    gradual_buyer = 'buyer_SYNTH_006'
    gradual_main_supplier = 'supplier_SYNTH_011'
    
    # H1: Normal, diverse (30 tenders, Jan-Jun)
    for i in range(30):
        supplier = f'supplier_SYNTH_OTHER_{i + 100}'
        n_tend = np.random.randint(3, 6)
        base_val = np.random.uniform(100_000, 500_000)
        discount = np.random.uniform(0.05, 0.15)
        day_offset = np.random.randint(0, 181)  # Jan-Jun
        pub_date = pd.Timestamp('2023-01-01') + pd.Timedelta(days=int(day_offset))
        
        t = make_tender(
            buyer_id=gradual_buyer, supplier_id=supplier,
            tender_value=base_val, award_value=base_val * (1 - discount),
            n_tenderers=n_tend, is_single=0, is_competitive=1,
            method='open', cpv=33.0,
            published_date=pub_date,
            scenario='6_gradual_capture'
        )
        all_tenders.append(t)
        
        # Create bids
        all_bids.append(make_bid(t['tender_id'], supplier, base_val * (1 - discount), True))
        for k in range(n_tend - 1):
            all_bids.append(make_bid(
                t['tender_id'], f'supplier_random_grad_{i}_{k}',
                base_val * np.random.uniform(0.90, 1.10), False
            ))
    
    # H2: Concentrated (30 tenders, Jul-Dec)
    for i in range(30):
        day_offset = np.random.randint(181, 365)  # Jul-Dec
        pub_date = pd.Timestamp('2023-01-01') + pd.Timedelta(days=int(day_offset))
        base_val = np.random.uniform(100_000, 500_000)
        
        if i < 18:  # 60% single bidder, all to main supplier
            supplier = gradual_main_supplier
            n_tend = 1
            is_single = 1
            is_comp = 0
            method = 'limited'
            discount = np.random.uniform(0.01, 0.05)
        elif i < 24:  # 20% competitive but still main supplier
            supplier = gradual_main_supplier
            n_tend = np.random.randint(2, 4)
            is_single = 0
            is_comp = 1
            method = 'open'
            discount = np.random.uniform(0.03, 0.08)
        else:  # 20% other suppliers
            supplier = f'supplier_SYNTH_OTHER_{i + 200}'
            n_tend = np.random.randint(2, 4)
            is_single = 0
            is_comp = 1
            method = 'open'
            discount = np.random.uniform(0.05, 0.15)
        
        t = make_tender(
            buyer_id=gradual_buyer, supplier_id=supplier,
            tender_value=base_val, award_value=base_val * (1 - discount),
            n_tenderers=n_tend, is_single=is_single, is_competitive=is_comp,
            method=method, cpv=33.0,
            has_enquiries=1 if is_comp else 0,
            published_date=pub_date,
            scenario='6_gradual_capture'
        )
        all_tenders.append(t)
        
        # Create bids
        all_bids.append(make_bid(t['tender_id'], supplier, base_val * (1 - discount), True))
        if n_tend > 1:
            for k in range(n_tend - 1):
                all_bids.append(make_bid(
                    t['tender_id'], f'supplier_random_grad2_{i}_{k}',
                    base_val * np.random.uniform(1.0, 1.15), False
                ))
    
    # =========================================================================
    # Build DataFrames
    # =========================================================================
    tenders_df = pd.DataFrame(all_tenders)
    bids_df = pd.DataFrame(all_bids)
    
    print(f"\nGenerated: {len(tenders_df)} synthetic tenders, {len(bids_df)} synthetic bids")
    print(f"Scenarios: {tenders_df['scenario'].value_counts().to_dict()}")
    
    return tenders_df, bids_df


synthetic_tenders, synthetic_bids = generate_synthetic_tenders()
synthetic_tenders.head()

Generating Scenario 1a: Naive Cartel...
Generating Scenario 1b: Sophisticated Cartel...
Generating Scenario 2: Monopolist Buyer...
Generating Scenario 3: Weekend/Price Manipulator...
Generating Scenario 4: Contract Splitter...
Generating Scenario 5: Exclusive High-Value Pair...
Generating Scenario 6: Gradual Capture...

Generated: 480 synthetic tenders, 1053 synthetic bids
Scenarios: {'4_contract_splitter': 150, '2_monopolist_buyer': 80, '3_weekend_price_manipulator': 60, '6_gradual_capture': 60, '1a_cartel_naive': 50, '1b_cartel_sophisticated': 50, '5_exclusive_high_value': 30}


,tender_id,ocid,buyer_id,procuring_entity_id,supplier_id,locality,postal_code,procurement_method,main_procurement_category,award_criteria,...,award_value_min,award_value_mean,award_value_std,award_concentration,discount_percentage_avg,discount_percentage_max,published_date,award_date,is_synthetic,scenario
0,SYNTH_T_00001,ocds-synth-SYNTH_T_00001,buyer_SYNTH_001,buyer_SYNTH_001,supplier_SYNTH_001,SYNTH_CITY,00000,open,goods,lowestCost,...,9.973511e+05,9.973511e+05,0.0,1.0,3.0,3.0,2023-12-15,2023-12-28,True,1a_cartel_naive
1,SYNTH_T_00002,ocds-synth-SYNTH_T_00002,buyer_SYNTH_001,buyer_SYNTH_001,supplier_SYNTH_002,SYNTH_CITY,00000,open,goods,lowestCost,...,6.596539e+05,6.596539e+05,0.0,1.0,3.0,3.0,2023-05-11,2023-05-29,True,1a_cartel_naive
2,SYNTH_T_00003,ocds-synth-SYNTH_T_00003,buyer_SYNTH_001,buyer_SYNTH_001,supplier_SYNTH_003,SYNTH_CITY,00000,open,goods,lowestCost,...,1.365598e+06,1.365598e+06,0.0,1.0,3.0,3.0,2023-09-10,2023-09-28,True,1a_cartel_naive
3,SYNTH_T_00004,ocds-synth-SYNTH_T_00004,buyer_SYNTH_001,buyer_SYNTH_001,supplier_SYNTH_004,SYNTH_CITY,00000,open,goods,lowestCost,...,6.974294e+05,6.974294e+05,0.0,1.0,3.0,3.0,2023-06-16,2023-07-01,True,1a_cartel_naive
4,SYNTH_T_00005,ocds-synth-SYNTH_T_00005,buyer_SYNTH_001,buyer_SYNTH_001,supplier_SYNTH_005,SYNTH_CITY,00000,open,goods,lowestCost,...,1.308638e+06,1.308638e+06,0.0,1.0,3.0,3.0,2023-09-21,2023-10-01,True,1a_cartel_naive


In [3]:
# Cell 3: Inject into Real Data

# Mark real data
real_tenders['is_synthetic'] = False
real_tenders['scenario'] = ''
real_bids['is_synthetic'] = False

# Ensure date column types match
if hasattr(real_tenders['published_date'].dtype, 'tz'):
    synthetic_tenders['published_date'] = synthetic_tenders['published_date'].dt.tz_localize('UTC')
    synthetic_tenders['award_date'] = synthetic_tenders['award_date'].dt.tz_localize('UTC')

# Align columns — keep only columns that exist in real data
common_cols = [c for c in real_tenders.columns if c in synthetic_tenders.columns]
tenders_combined = pd.concat([
    real_tenders[common_cols],
    synthetic_tenders[common_cols]
], ignore_index=True)

common_bid_cols = [c for c in real_bids.columns if c in synthetic_bids.columns]
bids_combined = pd.concat([
    real_bids[common_bid_cols],
    synthetic_bids[common_bid_cols]
], ignore_index=True)

print(f"Combined tenders: {len(tenders_combined):,} (real: {len(real_tenders):,}, synthetic: {len(synthetic_tenders):,})")
print(f"Combined bids: {len(bids_combined):,} (real: {len(real_bids):,}, synthetic: {len(synthetic_bids):,})")
print(f"\nSynthetic ratio: {len(synthetic_tenders)/len(tenders_combined)*100:.2f}%")

Combined tenders: 524,242 (real: 523,762, synthetic: 480)
Combined bids: 74,359 (real: 73,306, synthetic: 1,053)

Synthetic ratio: 0.09%


In [4]:
# Cell 4: Run Rule-Based Detector
print("="*60)
print("LEVEL 1: RULE-BASED DETECTION")
print("="*60)

rule_detector = RuleBasedDetector()
rule_results = rule_detector.detect(tenders_combined, bids_df=bids_combined, buyers_df=buyers)

# Check which synthetic tenders were flagged
synthetic_mask = rule_results['is_synthetic'] == True
synth_rule_results = rule_results[synthetic_mask].copy()

# Get all flag columns
flag_cols = [c for c in rule_results.columns if c.startswith('flag_')]

print(f"\n{'='*60}")
print("RULE-BASED RESULTS ON SYNTHETIC DATA")
print(f"{'='*60}")

# Results per scenario
rule_scenario_results = {}
for scenario in synth_rule_results['scenario'].unique():
    scenario_data = synth_rule_results[synth_rule_results['scenario'] == scenario]
    n = len(scenario_data)
    flagged = (scenario_data['rule_flags_count'] > 0).sum()
    avg_score = scenario_data['rule_risk_score'].mean()
    
    # Which rules triggered?
    triggered_rules = []
    for fc in flag_cols:
        count = scenario_data[fc].sum()
        if count > 0:
            triggered_rules.append((fc.replace('flag_', ''), int(count), round(count/n*100, 1)))
    triggered_rules.sort(key=lambda x: x[1], reverse=True)
    
    rule_scenario_results[scenario] = {
        'total': n,
        'flagged': flagged,
        'detection_rate': round(flagged / n * 100, 1),
        'avg_score': round(avg_score, 2),
        'triggered_rules': triggered_rules,
    }
    
    print(f"\n--- {scenario} ---")
    print(f"  Tenders: {n}, Flagged: {flagged} ({flagged/n*100:.1f}%), Avg score: {avg_score:.2f}")
    if triggered_rules:
        print(f"  Triggered rules:")
        for rule_name, cnt, pct in triggered_rules[:8]:
            print(f"    - {rule_name}: {cnt}/{n} ({pct}%)")

LEVEL 1: RULE-BASED DETECTION
Processing 524,242 tenders...
Step 1/4: Computing aggregations...
  Computing CPV stats...
  Computing buyer stats...
  Computing supplier stats...
  Computing pair stats...
  Aggregations complete.
Step 2/4: Merging reference data...
Step 3/4: Applying 45 rules...
  Applied 44 rules successfully.
Step 4/4: Computing risk levels and summary...
Detection complete!

RULE-BASED RESULTS ON SYNTHETIC DATA

--- 1a_cartel_naive ---
  Tenders: 50, Flagged: 50 (100.0%), Avg score: 2.80
  Triggered rules:
    - captive_supplier: 50/50 (100.0%)
    - close_to_winner: 34/50 (68.0%)
    - threshold_manipulation: 3/50 (6.0%)

--- 1b_cartel_sophisticated ---
  Tenders: 50, Flagged: 50 (100.0%), Avg score: 2.08
  Triggered rules:
    - captive_supplier: 50/50 (100.0%)
    - threshold_manipulation: 2/50 (4.0%)

--- 2_monopolist_buyer ---
  Tenders: 80, Flagged: 80 (100.0%), Avg score: 4.42
  Triggered rules:
    - buyer_supplier_dominance: 56/80 (70.0%)
    - captive_suppl

In [5]:
# Cell 5: Run Statistical Detector
print("="*60)
print("LEVEL 2: STATISTICAL DETECTION")
print("="*60)

stat_detector = StatisticalDetector()
stat_results = stat_detector.detect(tenders_combined, bids_df=bids_combined)

# Check synthetic tenders
synthetic_mask = stat_results['is_synthetic'] == True
synth_stat_results = stat_results[synthetic_mask].copy()

# Get all stat flag columns
stat_flag_cols = [c for c in stat_results.columns if c.startswith('stat_') and
                  c not in ['stat_score', 'stat_flags_count', 'stat_risk_level', 'stat_anomaly']]

print(f"\n{'='*60}")
print("STATISTICAL RESULTS ON SYNTHETIC DATA")
print(f"{'='*60}")

stat_scenario_results = {}
for scenario in synth_stat_results['scenario'].unique():
    scenario_data = synth_stat_results[synth_stat_results['scenario'] == scenario]
    n = len(scenario_data)
    flagged = (scenario_data['stat_flags_count'] > 0).sum()
    avg_score = scenario_data['stat_score'].mean()
    
    # Which stats triggered?
    triggered_stats = []
    for sc in stat_flag_cols:
        count = scenario_data[sc].sum()
        if count > 0:
            triggered_stats.append((sc.replace('stat_', ''), int(count), round(count/n*100, 1)))
    triggered_stats.sort(key=lambda x: x[1], reverse=True)
    
    stat_scenario_results[scenario] = {
        'total': n,
        'flagged': flagged,
        'detection_rate': round(flagged / n * 100, 1),
        'avg_score': round(avg_score, 2),
        'triggered_stats': triggered_stats,
    }
    
    print(f"\n--- {scenario} ---")
    print(f"  Tenders: {n}, Flagged: {flagged} ({flagged/n*100:.1f}%), Avg score: {avg_score:.2f}")
    if triggered_stats:
        print(f"  Triggered statistics:")
        for stat_name, cnt, pct in triggered_stats[:8]:
            print(f"    - {stat_name}: {cnt}/{n} ({pct}%)")

LEVEL 2: STATISTICAL DETECTION
Processing 524,242 tenders...
Step 1/5: Computing value outliers (Z-score, IQR)...
Step 2/5: Analyzing price patterns...
Step 3/5: Running Benford's Law analysis...
    Testing Benford per buyer...
    Buyers tested: 4,424, anomalies: 6
    Testing Benford per supplier...
    Suppliers tested: 16,190, anomalies: 43
Step 4/5: Analyzing bid spreads...
Step 5/5: Computing market concentration...
Statistical screening complete!

STATISTICAL RESULTS ON SYNTHETIC DATA

--- 1a_cartel_naive ---
  Tenders: 50, Flagged: 50 (100.0%), Avg score: 9.24
  Triggered statistics:
    - iqr_value: 50/50 (100.0%)
    - benford_buyer: 50/50 (100.0%)
    - benford_supplier: 50/50 (100.0%)
    - cv_anomaly: 50/50 (100.0%)
    - iqr_value_cpv: 47/50 (94.0%)
    - bid_clustering: 45/50 (90.0%)
    - ks_anomaly: 31/50 (62.0%)
    - rdnor_anomaly: 30/50 (60.0%)

--- 1b_cartel_sophisticated ---
  Tenders: 50, Flagged: 50 (100.0%), Avg score: 6.50
  Triggered statistics:
    - iqr_va

In [6]:
# Cell 6: Run ML Methods (Aggregated: IForest, LOF)
# NOTE: buyers_df=None forces aggregation from tenders_combined,
# so synthetic buyers are included (buyers.csv doesn't have them).
print("="*60)
print("LEVEL 3: ML DETECTION (Aggregated)")
print("="*60)

synthetic_buyer_ids = [
    'buyer_SYNTH_001', 'buyer_SYNTH_002', 'buyer_SYNTH_003',
    'buyer_SYNTH_004', 'buyer_SYNTH_005', 'buyer_SYNTH_006'
]

ml_results = {}

for algo_name in ['iforest', 'lof']:
    print(f"\n{'='*40}")
    print(f"Running {algo_name.upper()}...")
    print(f"{'='*40}")
    
    detector = AggregatedPyOD(algorithm=algo_name, contamination=0.05)
    
    # Buyer-level: aggregate from tenders (NOT pre-computed buyers.csv)
    print("\n--- Buyer-level (aggregated from tenders) ---")
    buyer_res = detector.detect_buyers(tenders_combined, buyers_df=None)
    
    synth_buyers = buyer_res[buyer_res['buyer_id'].isin(synthetic_buyer_ids)]
    print(f"\nSynthetic buyers detection:")
    for _, row in synth_buyers.iterrows():
        status = 'DETECTED' if row['anomaly'] == 1 else 'missed'
        print(f"  {row['buyer_id']}: score={row['score']:.3f}, anomaly={status}")
    
    # Pair-level
    print("\n--- Pair-level ---")
    pair_res = detector.detect_pairs(tenders_combined, min_contracts=3)
    
    if len(pair_res) > 0:
        synth_pairs = pair_res[
            pair_res['buyer_id'].str.startswith('buyer_SYNTH', na=False) |
            pair_res['supplier_id'].str.startswith('supplier_SYNTH', na=False)
        ]
        print(f"\nSynthetic pairs detection:")
        for _, row in synth_pairs.iterrows():
            status = 'DETECTED' if row['anomaly'] == 1 else 'missed'
            print(f"  {row['buyer_id']} <-> {row['supplier_id']}: "
                  f"score={row['score']:.3f}, anomaly={status}")
    
    ml_results[algo_name] = {
        'buyer_results': buyer_res,
        'pair_results': pair_res,
    }

LEVEL 3: ML DETECTION (Aggregated)

Running IFOREST...

--- Buyer-level (aggregated from tenders) ---
AggregatedPyOD (IFOREST): Detecting anomalous BUYERS...
  Computing buyer features from tenders...
  Features: ['single_bidder_rate', 'competitive_rate', 'avg_discount_pct', 'supplier_diversity_index', 'total_tenders', 'avg_value', 'total_value', 'cpv_concentration', 'avg_award_days', 'weekend_rate', 'value_variance_coeff', 'q4_rate']
  Buyers: 25,674
  Anomalies: 1,284 (5.0%)

Synthetic buyers detection:
  buyer_SYNTH_001: score=1.000, anomaly=DETECTED
  buyer_SYNTH_005: score=1.000, anomaly=DETECTED
  buyer_SYNTH_002: score=1.000, anomaly=DETECTED
  buyer_SYNTH_003: score=1.000, anomaly=DETECTED
  buyer_SYNTH_006: score=1.000, anomaly=DETECTED
  buyer_SYNTH_004: score=1.000, anomaly=DETECTED

--- Pair-level ---
AggregatedPyOD (IFOREST): Detecting anomalous PAIRS...
  Computing pair features from tenders...
  Pairs with 3+ contracts: 35,099
  Features: ['contracts_count', 'total_value

In [7]:
# Cell 7: Run Network Analysis
print("="*60)
print("LEVEL 4: NETWORK ANALYSIS")
print("="*60)

net_detector = NetworkAnalysisDetector(
    min_co_bids=3,
    min_contracts=3,
    suspicious_min_degree=5,       # Lower threshold to catch synthetic
    suspicious_min_clustering=0.5,
    rotation_min_ratio=0.5,
    rotation_min_interactions=3,
    monopoly_min_ratio=0.8,
    monopoly_min_contracts=10,
)

net_results = net_detector.fit_detect(tenders_combined, bids_df=bids_combined)

# Check synthetic tenders in network results
# Merge network results back with combined tenders to get is_synthetic
net_with_synth = net_results.merge(
    tenders_combined[['tender_id', 'is_synthetic', 'scenario']],
    on='tender_id', how='left'
)

synth_net = net_with_synth[net_with_synth['is_synthetic'] == True]

print(f"\n{'='*60}")
print("NETWORK RESULTS ON SYNTHETIC DATA")
print(f"{'='*60}")

net_scenario_results = {}
for scenario in synth_net['scenario'].unique():
    scenario_data = synth_net[synth_net['scenario'] == scenario]
    n = len(scenario_data)
    
    n_suspicious = scenario_data['network_suspicious_supplier'].sum()
    n_monopolistic = scenario_data['network_monopolistic'].sum()
    n_rotation = scenario_data['network_rotation'].sum()
    n_any = scenario_data['network_anomaly'].sum()
    
    net_scenario_results[scenario] = {
        'total': n,
        'network_any': int(n_any),
        'detection_rate': round(n_any / n * 100, 1) if n > 0 else 0,
        'suspicious_supplier': int(n_suspicious),
        'monopolistic': int(n_monopolistic),
        'rotation': int(n_rotation),
    }
    
    print(f"\n--- {scenario} ---")
    print(f"  Tenders: {n}")
    print(f"  Network anomaly: {n_any}/{n} ({n_any/n*100:.1f}%)")
    print(f"    - Suspicious supplier: {n_suspicious}")
    print(f"    - Monopolistic: {n_monopolistic}")
    print(f"    - Rotation: {n_rotation}")

# Check cartel communities
print(f"\n--- Cartel Communities ---")
cartel_candidates = net_detector.get_cartel_candidates(min_size=3)
synth_suppliers = set(f'supplier_SYNTH_{i:03d}' for i in range(1, 11))

for i, community in enumerate(cartel_candidates[:10]):
    synth_in_community = community & synth_suppliers
    if synth_in_community:
        print(f"  Community {i}: {len(community)} members, synthetic: {synth_in_community}")

# Check rotation pairs
print(f"\n--- Rotation Pairs ---")
rotation_df = net_detector.get_rotation_pairs(min_ratio=0.3)
if len(rotation_df) > 0:
    synth_rotation = rotation_df[
        rotation_df['bidder_1'].str.startswith('supplier_SYNTH', na=False) |
        rotation_df['bidder_2'].str.startswith('supplier_SYNTH', na=False)
    ]
    if len(synth_rotation) > 0:
        print(f"  Found {len(synth_rotation)} synthetic rotation pairs:")
        for _, row in synth_rotation.head(10).iterrows():
            print(f"    {row['bidder_1']} <-> {row['bidder_2']}: "
                  f"ratio={row['rotation_ratio']:.2f}, interactions={row['total_interactions']}")
    else:
        print("  No synthetic rotation pairs found.")

# Check monopolistic
print(f"\n--- Monopolistic Pairs ---")
mono_df = net_detector.get_monopolistic_relationships()
if len(mono_df) > 0:
    synth_mono = mono_df[
        mono_df['buyer_id'].str.startswith('buyer_SYNTH', na=False)
    ]
    if len(synth_mono) > 0:
        print(f"  Found {len(synth_mono)} synthetic monopolistic pairs:")
        for _, row in synth_mono.iterrows():
            print(f"    {row['buyer_id']} -> {row['supplier_id']}: "
                  f"dominance={row['dominance_ratio']:.2f}, contracts={row['contract_count']}")

LEVEL 4: NETWORK ANALYSIS
Processing 524,242 tenders...
  Competitive tenders: 16,046
  Bids in competitive: 8,731

Step 1/5: Building co-bidding network...
    Nodes: 56
    Edges: 63
Step 2/5: Detecting communities...
    Using igraph (fast)...
    Communities: 9
Step 3/5: Building winner-loser network...
    Nodes: 115
    Edges: 168
    Bid rotation pairs: 24
Step 4/5: Building buyer-supplier network...
    Nodes: 31,543
    Edges: 35,099
    Monopolistic pairs (>=80%, >=10 contracts): 146
Step 5/5: Skipping full collusion graph (disabled)

Computing tender-level results...
    Suspicious suppliers: 0
    Strict monopolistic pairs: 146
    Strict rotation pairs: 24

Network Analysis complete!
  Tenders with network flags: 12,161

NETWORK RESULTS ON SYNTHETIC DATA

--- 1a_cartel_naive ---
  Tenders: 50
  Network anomaly: 50/50 (100.0%)
    - Suspicious supplier: 0
    - Monopolistic: 0
    - Rotation: 50

--- 1b_cartel_sophisticated ---
  Tenders: 50
  Network anomaly: 50/50 (100.0%

In [8]:
# Cell 8: Honest Detection Matrix — only RELEVANT rules/stats per scenario
#
# Problem with naive approach: captive_supplier, iqr_value, benford flags
# trigger on ALL synthetic data as side effects, inflating detection rates.
# Here we define which rules/stats are actually relevant to each anomaly pattern.

print("="*70)
print("HONEST DETECTION MATRIX (only relevant signals)")
print("="*70)

# =========================================================================
# Define RELEVANT rules and stats per scenario
# =========================================================================
RELEVANT_RULES = {
    '1a_cartel_naive': [
        'bid_rotation', 'cobidding_same_winner', 'close_to_winner',
        'identical_bids', 'low_win_rate',
    ],
    '1b_cartel_sophisticated': [
        'bid_rotation', 'cobidding_same_winner', 'close_to_winner',
        'identical_bids', 'low_win_rate',
    ],
    '2_monopolist_buyer': [
        'single_bidder', 'buyer_supplier_dominance', 'single_bidder_low_discount',
        'no_enquiries', 'high_limited_usage',
    ],
    '3_weekend_price_manipulator': [
        'weekend_publication', 'extreme_discount', 'round_bid_prices',
    ],
    '4_contract_splitter': [
        'threshold_manipulation', 'contract_splitting', 'multiple_near_threshold',
        'direct_awards_pattern', 'same_day_same_supplier', 'high_limited_usage',
    ],
    '5_exclusive_high_value': [
        'single_bidder', 'single_bidder_low_discount', 'buyer_supplier_dominance',
        'high_market_share', 'price_outlier', 'value_zscore_outlier',
    ],
    '6_gradual_capture': [
        'single_bidder', 'single_bidder_low_discount', 'buyer_supplier_dominance',
        'no_enquiries', 'high_limited_usage',
    ],
}

RELEVANT_STATS = {
    '1a_cartel_naive': [
        'cv_anomaly', 'bid_clustering', 'bid_spread_anomaly',
        'ks_anomaly', 'rdnor_anomaly', 'skewness_anomaly', 'kurtosis_anomaly',
        'diffp_anomaly',
    ],
    '1b_cartel_sophisticated': [
        'cv_anomaly', 'bid_clustering', 'bid_spread_anomaly',
        'ks_anomaly', 'rdnor_anomaly', 'skewness_anomaly', 'kurtosis_anomaly',
        'diffp_anomaly',
    ],
    '2_monopolist_buyer': [
        'high_concentration', 'monopoly',
    ],
    '3_weekend_price_manipulator': [
        'zscore_discount', 'iqr_discount', 'round_price', 'very_round_price',
    ],
    '4_contract_splitter': [
        'high_concentration', 'monopoly',
    ],
    '5_exclusive_high_value': [
        'zscore_value', 'zscore_value_cpv', 'iqr_value_cpv',
        'high_concentration', 'monopoly',
    ],
    '6_gradual_capture': [
        'high_concentration', 'monopoly',
    ],
}

RELEVANT_NETWORK = {
    '1a_cartel_naive': ['network_rotation', 'network_suspicious_supplier'],
    '1b_cartel_sophisticated': ['network_rotation', 'network_suspicious_supplier'],
    '2_monopolist_buyer': ['network_monopolistic'],
    '3_weekend_price_manipulator': [],  # No network pattern expected
    '4_contract_splitter': ['network_monopolistic'],
    '5_exclusive_high_value': ['network_monopolistic'],
    '6_gradual_capture': ['network_monopolistic'],
}

# Which scenarios should ML detect? (aggregated buyer/pair patterns)
ML_EXPECTED = {
    '1a_cartel_naive': 'pair',          # Cartel pairs, not buyer-level
    '1b_cartel_sophisticated': 'pair',  # Same
    '2_monopolist_buyer': 'both',       # Both buyer and pair
    '3_weekend_price_manipulator': None, # Tender-level pattern, not aggregated
    '4_contract_splitter': 'both',      # High volume buyer + exclusive pair
    '5_exclusive_high_value': 'both',   # Extreme buyer + exclusive pair
    '6_gradual_capture': 'both',        # Buyer portrait + pair with main supplier
}

# =========================================================================
# Compute honest detection rates
# =========================================================================

def compute_relevant_rule_rate(scenario_key, synth_rule_data):
    """% of tenders flagged by at least one RELEVANT rule."""
    relevant = RELEVANT_RULES.get(scenario_key, [])
    if not relevant:
        return None
    relevant_cols = [f'flag_{r}' for r in relevant if f'flag_{r}' in synth_rule_data.columns]
    if not relevant_cols:
        return 0.0
    flagged = (synth_rule_data[relevant_cols].sum(axis=1) > 0).sum()
    return round(flagged / len(synth_rule_data) * 100, 1)

def compute_relevant_stat_rate(scenario_key, synth_stat_data):
    """% of tenders flagged by at least one RELEVANT stat."""
    relevant = RELEVANT_STATS.get(scenario_key, [])
    if not relevant:
        return None
    relevant_cols = [f'stat_{s}' for s in relevant if f'stat_{s}' in synth_stat_data.columns]
    if not relevant_cols:
        return 0.0
    flagged = (synth_stat_data[relevant_cols].sum(axis=1) > 0).sum()
    return round(flagged / len(synth_stat_data) * 100, 1)

def compute_relevant_network_rate(scenario_key, synth_net_data):
    """% of tenders flagged by relevant network signal."""
    relevant = RELEVANT_NETWORK.get(scenario_key, [])
    if not relevant:
        return None  # Not expected
    relevant_cols = [c for c in relevant if c in synth_net_data.columns]
    if not relevant_cols:
        return 0.0
    flagged = (synth_net_data[relevant_cols].sum(axis=1) > 0).sum()
    return round(flagged / len(synth_net_data) * 100, 1)

def check_ml_buyer(algo_name, buyer_ids):
    """Check buyer-level ML detection."""
    res = ml_results.get(algo_name, {})
    buyer_res = res.get('buyer_results')
    if buyer_res is None or len(buyer_res) == 0:
        return None
    synth_b = buyer_res[buyer_res['buyer_id'].isin(buyer_ids)]
    anomaly_col = 'anomaly' if 'anomaly' in synth_b.columns else 'is_anomaly'
    if len(synth_b) == 0:
        return False
    return synth_b[anomaly_col].sum() > 0

def check_ml_pair(algo_name, pair_buyer_ids, pair_supplier_ids):
    """Check pair-level ML detection."""
    res = ml_results.get(algo_name, {})
    pair_res = res.get('pair_results')
    if pair_res is None or len(pair_res) == 0:
        return None
    for b_id, s_id in zip(pair_buyer_ids, pair_supplier_ids or [None]*len(pair_buyer_ids)):
        mask = pair_res['buyer_id'] == b_id
        if s_id:
            mask = mask & (pair_res['supplier_id'] == s_id)
        synth_p = pair_res[mask]
        anomaly_col = 'anomaly' if 'anomaly' in synth_p.columns else 'is_anomaly'
        if len(synth_p) > 0 and synth_p[anomaly_col].sum() > 0:
            return True
    return False

# =========================================================================
# Build honest detection matrix
# =========================================================================
scenarios = {
    '1a_cartel_naive': {
        'name': '1a. Naive Cartel',
        'buyer_ids': ['buyer_SYNTH_001'],
        'pair_buyer_ids': ['buyer_SYNTH_001'],
        'pair_supplier_ids': [None],
    },
    '1b_cartel_sophisticated': {
        'name': '1b. Sophisticated Cartel',
        'buyer_ids': ['buyer_SYNTH_001'],
        'pair_buyer_ids': ['buyer_SYNTH_001'],
        'pair_supplier_ids': [None],
    },
    '2_monopolist_buyer': {
        'name': '2. Monopolist Buyer',
        'buyer_ids': ['buyer_SYNTH_002'],
        'pair_buyer_ids': ['buyer_SYNTH_002'],
        'pair_supplier_ids': ['supplier_SYNTH_006'],
    },
    '3_weekend_price_manipulator': {
        'name': '3. Weekend/Price Manip',
        'buyer_ids': ['buyer_SYNTH_003'],
        'pair_buyer_ids': ['buyer_SYNTH_003', 'buyer_SYNTH_003'],
        'pair_supplier_ids': ['supplier_SYNTH_007', 'supplier_SYNTH_008'],
    },
    '4_contract_splitter': {
        'name': '4. Contract Splitter',
        'buyer_ids': ['buyer_SYNTH_004'],
        'pair_buyer_ids': ['buyer_SYNTH_004'],
        'pair_supplier_ids': ['supplier_SYNTH_009'],
    },
    '5_exclusive_high_value': {
        'name': '5. Exclusive High-Value',
        'buyer_ids': ['buyer_SYNTH_005'],
        'pair_buyer_ids': ['buyer_SYNTH_005'],
        'pair_supplier_ids': ['supplier_SYNTH_010'],
    },
    '6_gradual_capture': {
        'name': '6. Gradual Capture',
        'buyer_ids': ['buyer_SYNTH_006'],
        'pair_buyer_ids': ['buyer_SYNTH_006'],
        'pair_supplier_ids': ['supplier_SYNTH_011'],
    },
}

detection_matrix = []

for scenario_key, scenario_info in scenarios.items():
    synth_mask = synth_rule_results['scenario'] == scenario_key
    synth_stat_mask = synth_stat_results['scenario'] == scenario_key
    synth_net_mask = synth_net['scenario'] == scenario_key
    
    row = {'Scenario': scenario_info['name']}
    
    # Rule-based: only relevant rules
    row['Rule-based'] = compute_relevant_rule_rate(
        scenario_key, synth_rule_results[synth_mask]
    )
    
    # Statistical: only relevant stats
    row['Statistical'] = compute_relevant_stat_rate(
        scenario_key, synth_stat_results[synth_stat_mask]
    )
    
    # ML: separate buyer and pair, only where expected
    ml_expected = ML_EXPECTED[scenario_key]
    
    for algo in ['IForest', 'LOF']:
        algo_key = algo.lower()
        if ml_expected is None:
            row[algo] = None  # Not expected for this scenario
        elif ml_expected == 'pair':
            row[algo] = check_ml_pair(
                algo_key, scenario_info['pair_buyer_ids'],
                scenario_info.get('pair_supplier_ids')
            )
        elif ml_expected == 'both':
            buyer_det = check_ml_buyer(algo_key, scenario_info['buyer_ids'])
            pair_det = check_ml_pair(
                algo_key, scenario_info['pair_buyer_ids'],
                scenario_info.get('pair_supplier_ids')
            )
            # Report as "buyer/pair" detail
            row[algo] = f"B:{'Y' if buyer_det else 'N'} P:{'Y' if pair_det else 'N'}"
        else:
            row[algo] = check_ml_buyer(algo_key, scenario_info['buyer_ids'])
    
    # Network: only relevant signals
    row['Network'] = compute_relevant_network_rate(
        scenario_key, synth_net[synth_net_mask]
    )
    
    detection_matrix.append(row)

detection_df = pd.DataFrame(detection_matrix)

# =========================================================================
# Format and display
# =========================================================================
def format_cell(val):
    if val is None:
        return 'n/a'
    if isinstance(val, bool):
        return 'Y' if val else 'N'
    if isinstance(val, str):
        return val
    if isinstance(val, (int, float)):
        return f'{val:.0f}%' if val > 0 else '0%'
    return str(val)

display_df = detection_df.copy()
for col in display_df.columns:
    if col != 'Scenario':
        display_df[col] = display_df[col].apply(format_cell)

print("\n" + display_df.to_string(index=False))

# =========================================================================
# Detailed breakdown per scenario
# =========================================================================
print(f"\n{'='*70}")
print("DETAILED BREAKDOWN")
print(f"{'='*70}")

for scenario_key, scenario_info in scenarios.items():
    print(f"\n--- {scenario_info['name']} ---")
    
    # Relevant rules that actually triggered
    synth_data = synth_rule_results[synth_rule_results['scenario'] == scenario_key]
    relevant = RELEVANT_RULES[scenario_key]
    n = len(synth_data)
    print(f"  Rule-based (relevant only):")
    any_relevant = False
    for r in relevant:
        col = f'flag_{r}'
        if col in synth_data.columns:
            cnt = int(synth_data[col].sum())
            if cnt > 0:
                print(f"    {r}: {cnt}/{n} ({cnt/n*100:.0f}%)")
                any_relevant = True
    if not any_relevant:
        print(f"    (none triggered)")
    
    # Relevant stats
    synth_stat_data = synth_stat_results[synth_stat_results['scenario'] == scenario_key]
    relevant_s = RELEVANT_STATS[scenario_key]
    print(f"  Statistical (relevant only):")
    any_relevant = False
    for s in relevant_s:
        col = f'stat_{s}'
        if col in synth_stat_data.columns:
            cnt = int(synth_stat_data[col].sum())
            if cnt > 0:
                print(f"    {s}: {cnt}/{n} ({cnt/n*100:.0f}%)")
                any_relevant = True
    if not any_relevant:
        print(f"    (none triggered)")
    
    # Network
    relevant_n = RELEVANT_NETWORK[scenario_key]
    if relevant_n:
        synth_net_data = synth_net[synth_net['scenario'] == scenario_key]
        print(f"  Network (relevant only):")
        for nc in relevant_n:
            if nc in synth_net_data.columns:
                cnt = int(synth_net_data[nc].sum())
                print(f"    {nc}: {cnt}/{n} ({cnt/n*100:.0f}%)")
    else:
        print(f"  Network: not expected for this pattern")

# =========================================================================
# Coverage summary
# =========================================================================
print(f"\n{'='*70}")
print("DETECTION COVERAGE (honest)")
print(f"{'='*70}")

for _, row in detection_df.iterrows():
    methods_detected = 0
    method_names = []
    for col in ['Rule-based', 'Statistical', 'IForest', 'LOF', 'Network']:
        val = row[col]
        if val is None:
            continue  # n/a = not expected
        if isinstance(val, bool) and val:
            methods_detected += 1
            method_names.append(col)
        elif isinstance(val, str) and 'Y' in val:
            methods_detected += 1
            method_names.append(col)
        elif isinstance(val, (int, float)) and not isinstance(val, bool) and val > 0:
            methods_detected += 1
            method_names.append(col)
    
    # Count expected methods (non-None)
    expected = sum(1 for col in ['Rule-based', 'Statistical', 'IForest', 'LOF', 'Network']
                   if row[col] is not None)
    print(f"  {row['Scenario']}: {methods_detected}/{expected} expected methods [{', '.join(method_names)}]")

HONEST DETECTION MATRIX (only relevant signals)

                Scenario Rule-based Statistical IForest     LOF Network
        1a. Naive Cartel        68%        100%       Y       N    100%
1b. Sophisticated Cartel         0%         84%       Y       N    100%
     2. Monopolist Buyer       100%        100% B:Y P:Y B:N P:Y      0%
  3. Weekend/Price Manip       100%        100%     n/a     n/a      0%
    4. Contract Splitter       100%        100% B:Y P:Y B:Y P:Y    100%
 5. Exclusive High-Value       100%        100% B:Y P:Y B:Y P:Y    100%
      6. Gradual Capture         8%          0% B:Y P:Y B:N P:Y      0%

DETAILED BREAKDOWN

--- 1a. Naive Cartel ---
  Rule-based (relevant only):
    close_to_winner: 34/50 (68%)
  Statistical (relevant only):
    cv_anomaly: 50/50 (100%)
    bid_clustering: 45/50 (90%)
    ks_anomaly: 31/50 (62%)
    rdnor_anomaly: 30/50 (60%)
    skewness_anomaly: 21/50 (42%)
  Network (relevant only):
    network_rotation: 50/50 (100%)
    network_suspici

In [9]:
# Cell 9: Save Results
import os

results_dir = os.path.join('..', 'results')
os.makedirs(results_dir, exist_ok=True)

# Save detection matrix
output_path = os.path.join(results_dir, 'synthetic_validation_results.csv')
detection_df.to_csv(output_path, index=False)
print(f"Saved detection matrix to: {output_path}")

# Save detailed per-scenario results (ONLY relevant rules/stats)
detailed_rows = []
for scenario_key, scenario_info in scenarios.items():
    rule_res = rule_scenario_results.get(scenario_key, {})
    stat_res = stat_scenario_results.get(scenario_key, {})
    net_res = net_scenario_results.get(scenario_key, {})
    
    # Filter triggered rules to RELEVANT only
    relevant_rules = set(RELEVANT_RULES.get(scenario_key, []))
    all_triggered_rules = rule_res.get('triggered_rules', [])
    relevant_triggered_rules = [(r, cnt, pct) for r, cnt, pct in all_triggered_rules
                                if r in relevant_rules]
    
    # Filter triggered stats to RELEVANT only
    relevant_stats = set(RELEVANT_STATS.get(scenario_key, []))
    all_triggered_stats = stat_res.get('triggered_stats', [])
    relevant_triggered_stats = [(s, cnt, pct) for s, cnt, pct in all_triggered_stats
                                if s in relevant_stats]
    
    # Honest detection rates (from Cell 8 detection_df)
    scenario_row = detection_df[detection_df['Scenario'] == scenario_info['name']].iloc[0]
    
    detailed_rows.append({
        'scenario': scenario_key,
        'scenario_name': scenario_info['name'],
        'n_tenders': rule_res.get('total', 0),
        # Rule-based (honest: only relevant rules)
        'rule_detection_rate': scenario_row['Rule-based'],
        'rule_avg_score': rule_res.get('avg_score', 0),
        'rule_relevant_triggered': '; '.join(
            f"{r}({pct}%)" for r, cnt, pct in relevant_triggered_rules
        ) if relevant_triggered_rules else '(none)',
        # Statistical (honest: only relevant stats)
        'stat_detection_rate': scenario_row['Statistical'],
        'stat_avg_score': stat_res.get('avg_score', 0),
        'stat_relevant_triggered': '; '.join(
            f"{s}({pct}%)" for s, cnt, pct in relevant_triggered_stats
        ) if relevant_triggered_stats else '(none)',
        # ML (from honest detection matrix)
        'iforest_detected': scenario_row['IForest'],
        'lof_detected': scenario_row['LOF'],
        # Network (honest: only relevant signals)
        'network_detection_rate': scenario_row['Network'],
        'network_suspicious': net_res.get('suspicious_supplier', 0),
        'network_monopolistic': net_res.get('monopolistic', 0),
        'network_rotation': net_res.get('rotation', 0),
    })

detailed_df = pd.DataFrame(detailed_rows)
detailed_path = os.path.join(results_dir, 'synthetic_validation_detailed.csv')
detailed_df.to_csv(detailed_path, index=False)
print(f"Saved detailed results to: {detailed_path}")

# Print final summary
print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
print(f"Total synthetic tenders: {len(synthetic_tenders)}")
print(f"Total synthetic bids: {len(synthetic_bids)}")
print(f"Scenarios: {len(scenarios)}")
print(f"Methods tested: 5 (Rule-based, Statistical, IForest, LOF, Network)")
print(f"\nDetailed CSV now contains ONLY relevant rules/stats per scenario.")
print(f"Side-effect flags (captive_supplier, iqr_value, benford) excluded.")

Saved detection matrix to: ..\results\synthetic_validation_results.csv
Saved detailed results to: ..\results\synthetic_validation_detailed.csv

SUMMARY
Total synthetic tenders: 480
Total synthetic bids: 1053
Scenarios: 7
Methods tested: 5 (Rule-based, Statistical, IForest, LOF, Network)

Detailed CSV now contains ONLY relevant rules/stats per scenario.
Side-effect flags (captive_supplier, iqr_value, benford) excluded.
